In [1]:
import pandas as pd
from glob import glob as gb
import os
from fuzzywuzzy import fuzz
import wikipedia
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup as bs
from fuzzywuzzy import process
import itertools
from fuzzywuzzy import fuzz
from collections import Counter
import random  
from datetime import datetime
import json 
import time

def rt(fn):
    with open(fn,'r') as f:
        c = f.read()
    s = bs(c)
    return s

DATAPATH = "/media/ruben/Elements/PhD/data/hansard"
list_files = gb(DATAPATH + '/raw/pm/members/*')

In [2]:
def find_name_full(soup):
    names = soup.find_all('name')
    names = [x for x in names if 'full' in [c.name for c in x.findChildren()]]
    if len(names) >= 1:
        return names[0]('full')[0].text
    else:
        return "na"

def find_name_comp(soup):
    names = soup.find_all('name')
    names = [x for x in names if 'last' in [c.name for c in x.findChildren()]]
    if len(names) >= 1:
        return names[0]('first')[0].text + " " + names[0]('last')[0].text

def clean_name(name):
    if name == None:
        return "na"
    name = name.lower()
    blacklist = "lord viscount baroness baron the marquess lieut.-colonel lieutenant lieut colonel captain hon.".split(' ')

    for x in blacklist:
        name = name.replace(x,'')
    if name[0] == ' ':
        name = name[1:]
    return name

In [4]:
d = []

for fn in tqdm(list_files):

    soup = rt(fn).find('member')
    member_id = soup.attrs['pm:id']

    try:
        full_name = find_name_full(soup)
    except:
        full_name = "na"

    try:
        comp_name = find_name_comp(soup)
    except:
        comp_name = "na"

    for mbs in soup.find_all('membership'):

        try:
            name = mbs.find('name').text
        except:
            name = "na"

        try:
            start = mbs.find('period').attrs['pm:from']
            end = mbs.find('period').attrs['pm:till']
        except:
            start = "na"
            end = "na"

        t = [member_id,full_name,clean_name(full_name), comp_name,clean_name(comp_name),name,clean_name(name), start, end]
        for i in ["pm:body","pm:membership-id","pm:district","pm:party-name"]:
            try:
                t.append(mbs.attrs[i])
            except:
                t.append("na")
        d.append(t)
    

100%|██████████| 13167/13167 [00:54<00:00, 243.31it/s]


In [5]:
df = pd.DataFrame(d,columns="id full_name full_name_cleaned comp_name comp_name_cleaned memb_name memb_name_clean start end body membership_id district party_name".split(' ')).fillna("na").replace("unknown","na")

In [10]:
party_correction = {'Alliance':'Alliance Party of Northern Ireland',
 'CWM':'CWM',
 'Con':'Conservative Party',
 'DCWM':'DCWM',
 'DUP':'Democratic Unionist Party',
 'Green':'Green Party of England and Wales',
 'Ind':'Independent',
 'Ind Con':'Independent Conservative',
 'Ind Lab':'Independent Labour',
 'Ind UU':'Independent',
 'LDem':'Liberal Democrats',
 'Lab':'Labour Party',
 'Lab/Co-op':'Labour Co-operative',
 'PC':'Plaid Cymru',
 'Res':'Respect Party',
 'SDLP':'Social Democratic and Labour Party',
 'SF':'Sinn Féin',
 'SNP':'Scottish National Party',
 'SPK':'Speaker of the House of Commons',
 'UKIP':'UK Independence Party',
 'UKUP':'UK Unionist Party',
 'UUP':'Ulster Unionist Party',
 'na':'na'}

In [12]:
df['party_name'] = [party_correction[c] for c in df['party_name']]

In [13]:
df.to_csv(DATAPATH + '/raw/pm/members_flattened.csv',index=False,sep='\t')